In [2]:
%run ../utils/commonImports.py
%run ../utils/tradingImports.py
%matplotlib inline

# Load Data

In [3]:
def calculate_EMA(history, col, time_period):
    return calculateTalib('EMA', {col: history[col].values}, {'timeperiod': time_period})


def add_EMA(df):
    df['blue'] = calculate_EMA(df, 'close', 8)
    df['green'] = calculate_EMA(df, 'close', 13)
    df['yellow'] = calculate_EMA(df, 'close', 21)
    df['red'] = calculate_EMA(df, 'close', 55)
    return df

In [4]:
def load_and_divide(db, table, from_date=None, to_date=None, train_ratio=0.5):
    data = load_trading_data(db, table, from_date, to_date)
    data = add_EMA(data)
    data_train, data_test = divide_train_and_test(data, train_ratio)
    return data, data_train, data_test


db = 'D:\\Dropbox\\My work\\krypl-project\\sqlite\\ploniex-chart-data\\USDT_BTC.db'
data, data_train, data_test = load_and_divide(db, 'chart_data', from_date='2017-01-01')
ohlc, ohlc_train, ohlc_test = data.copy(), data_train.copy(), data_test.copy()


features = ['red', 'yellow', 'green', 'blue']
data_manager_train = CurrencyDataManager(data_train['close'], data_train[features])
data_manager_test = CurrencyDataManager(data_test['close'], data_test[features])
wallet = {'usdt': 1000}
contract_pair = ContractPair.new('usdt', 'btc')

# Explore

In [5]:
correct_pattern = ohlc_train.query("red < yellow < green < blue").copy()

In [6]:
correct_pattern.shape

(5904, 14)

In [ ]:
def ratio(df, c1, c2):
    return (df[c1] / df[c2]) - 1


def distplot(df, col, x_max):
    plt.figure(figsize=[12, 6])
    plt.title(col, fontproperties=title_font)
    sns.distplot(df[col])
    plt.xlim([0, x_max])

correct_pattern['ratioRY'] = ratio(correct_pattern, 'yellow', 'red')
correct_pattern['ratioYG'] = ratio(correct_pattern, 'green', 'yellow')
correct_pattern['ratioGB'] = ratio(correct_pattern, 'blue', 'green')

In [ ]:
distplot(correct_pattern, 'ratioRY', 0.015)
distplot(correct_pattern, 'ratioYG', 0.005)
distplot(correct_pattern, 'ratioGB', 0.005)

# Implementation

In [7]:
from modeling.strategy import Strategy

class HoldStrategy(Strategy):
    def __init__(self, exchange, data_manager, contract_pair, trade_size):
        super().__init__(exchange, data_manager, contract_pair, trade_size, 0., 0.)    
    
    def trade(self):
        while self.data_manager.has_tick():            
            history, price = self.data_manager.tick(1)
            
            if history.shape[0] == 0:
                self.buy(price)
            else:
                continue
        
        if self.opened:
            self.sell_all(price)

In [8]:
from modeling.strategy import Strategy
from itertools import permutations

ORDERS = list(permutations('BGYR', 4))


class CustomStrategy(Strategy):
    def __init__(self, exchange, data_manager, contract_pair, trade_size, willing_loss, target_profit, params):
        super().__init__(exchange, data_manager, contract_pair, trade_size, willing_loss, target_profit)
        self.history_len = 1
        self.params = params
    
    def has_correct_order(self, _type, red, yellow, green, blue):
        order = self.params['order_' + _type]
        d = {'R': red, 'Y': yellow, 'G': green, 'B': blue}
        l = [d[order[i]] for i in range(len(order))]
        for i in range(len(l)-1):
            if l[i] < l[i+1]:
                return False
        
        thresholds = [self.params[f'r{i}_{_type}'] for i in range(1, len(order))]
        
        def ratio(x, y):
            return (x / y) - 1
        
        for i in range(len(l)-1):
            if ratio(l[i], l[i+1]) > thresholds[i]:
                return False
        
        return True
    
    
    def trade(self):
        after_sell = False
        while self.data_manager.has_tick():            
            history, price = self.data_manager.tick(self.history_len)
            
            if history.shape[0] == 0:
                continue
            
            red, yellow, green, blue = history[0]
            if np.isnan(red):
                continue
            
            if not self.opened and not after_sell and self.has_correct_order('buy', red, yellow, green, blue):
                self.buy(price)
                price_bought = price
            elif self.opened and self.is_risky(price_bought, price):
                self.sell_all(price)
                after_sell = True
            elif self.opened and self.is_target_satisfied(price_bought, price):
                self.sell_all(price)
                after_sell = True
            elif self.opened and self.has_correct_order('sell', red, yellow, green, blue):
                self.sell_all(price)
                after_sell = True
        
        if self.opened:
            self.sell_all(price)

In [9]:
def create_strategy(data_manager_p, param_array):
    params = {
        'order_buy': ORDERS[int(param_array[0])],
        'r1_buy': param_array[1],
        'r2_buy': param_array[2],
        'r3_buy': param_array[3],
        'order_sell': ORDERS[int(param_array[4])],
        'r1_sell': param_array[5],
        'r2_sell': param_array[6],
        'r3_sell': param_array[7],
    }
    
#     params = {
#         'order_buy': ORDERS[0],
#         'r1_buy': param_array[0],
#         'r2_buy': param_array[1],
#         'r3_buy': param_array[2],
#         'order_sell': ORDERS[7],
#         'r1_sell': param_array[3],
#         'r2_sell': param_array[4],
#         'r3_sell': param_array[5],
#     }
    
    data_manager = deepcopy(data_manager_p)
    exchange = BackTestExchange(data_manager, deepcopy(wallet), 0.0025)
    strategy = CustomStrategy(exchange, data_manager, contract_pair, 100, willing_loss=0.05, target_profit=1, params=params)
    return data_manager, exchange, strategy


def hold_stats(data_manager_p):
    data_manager = deepcopy(data_manager_p)
    exchange = BackTestExchange(data_manager, deepcopy(wallet), 0.0025)
    strategy = HoldStrategy(exchange, data_manager, contract_pair, 100)
    strategy.trade()
    return strategy.stats('usdt').report()

# Learning

In [10]:
def create_train_strategy(param_array):
    return create_strategy(data_manager_train, param_array)

In [11]:
hold_stats_train = hold_stats(data_manager_train)
hold_stats_train

,usdt
startAmount,"1,000.0000"
numberOfTrades,1.0000
totalProfit,282.2111
avgProfit,282.2111
winPercentage,100.0000
avgWinTrade,282.2111
avgLossTrade,0.0000
profitFactor,inf
maxDrawdown,0.0000


In [ ]:
data_manager, exchange, strategy = create_train_strategy([0, 0.005, 0.001, 0.0015, 10, 0.005, 0.001, 0.0015,])
strategy.trade()
r = strategy.stats('usdt').report()
r

In [27]:
from random import Random
from time import time
from math import cos
from math import pi
from inspyred import ec
from inspyred.ec import terminators

MIN_FLOAT = 1 / 10**5
MAX_FLOAT = 1-MIN_FLOAT

def random_args_part(random):
    l = [random.randint(0, len(ORDERS)-1)]
    l += [random.uniform(MIN_FLOAT, MAX_FLOAT) for _ in range(3)]
    return l


def generate_rastrigin(random, args):
    return random_args_part(random) + random_args_part(random)


def bound_args(candidate, args):
    for i in [0, 4]:
        candidate[i] = int(max(min(candidate[i], len(ORDERS)-1), 0))
    
    for i in [1, 2, 3, 5, 6, 7]:
        candidate[i] = max(min(candidate[i], MAX_FLOAT), MIN_FLOAT)
        
    return candidate

import itertools

# bound_args.lower_bound = itertools.repeat(MIN_FLOAT)
# bound_args.upper_bound = itertools.repeat(MAX_FLOAT)
    

def evaluate_rastrigin(candidates, args):
    fitness = []
    for cs in candidates:
        fit = f(cs)
        fitness.append(fit)
    return fitness


from scipy.optimize import minimize, brute
import sys
from math import log10

def optim_val(stats, hold_stats):
    return (stats.loc['totalProfit', 'usdt']) * (stats.loc['numberOfTrades', 'usdt'])

def f(x):
    data_manager, exchange, strategy = create_train_strategy(x)
    strategy.trade()
    r = strategy.stats('usdt').report()
    optimization_value = optim_val(r, hold_stats_train)
    sys.stdout.write(f"\r{x}, {optimization_value}")
    sys.stdout.flush()
    return optimization_value


rand = Random()
rand.seed(int(time()))
es = ec.ES(rand)
es.terminator = terminators.evaluation_termination
final_pop = es.evolve(generator=generate_rastrigin,
                      evaluator=evaluate_rastrigin,
                      pop_size=100,
                      maximize=True,
                      bounder=bound_args,
                      max_evaluations=200,
                      mutation_rate=0.25)
# Sort and print the best individual, who will be at index 0.
final_pop.sort(reverse=True)
print(final_pop[0])

[3, 1e-05, 0.10631719939762785, 0.28827554721715715, 18, 0.99999, 0.47421197564537326, 0.24267735776742053], 0.0[1, 0.20395837382277912, 1e-05, 0.99999, 10, 0.41907843807101913, 0.6511499959514674, 0.41645177891322194, 0.35190437971857796, 0.5343515611583227, 0.7211649462254905, 0.38951969087903626, 0.5146169403249824, 0.18470104519233443, 0.042603165740063, 0.1546125928340332] : 101.26169461800328


In [32]:
final_pop[-1].candidate

[3,
 0.837072713719854,
 0.6780691753414513,
 0.3763048270750474,
 15,
 0.4915319529410493,
 0.6954156098077054,
 0.7953752768496755,
 0.3111630966516624,
 0.855396491886018,
 0.07616869280063976,
 0.9132715215021304,
 0.41162958546311434,
 0.7894776458342089,
 0.7284303204327756,
 0.5199314052092769]

In [19]:
data_manager, exchange, strategy = create_train_strategy(final_pop[0].candidate)
strategy.trade()
r = strategy.stats('usdt').report()
print(optim_val(r, hold_stats_train))
r

100.95492065790813


,usdt
startAmount,"1,000.0000"
numberOfTrades,1.0000
totalProfit,100.9549
avgProfit,100.9549
winPercentage,100.0000
avgWinTrade,100.9549
avgLossTrade,0.0000
profitFactor,inf
maxDrawdown,0.0000


# Look on transactions

In [ ]:
len(strategy.exchange.transactions)

In [ ]:
%matplotlib notebook
plot_transactions(ohlc_train, strategy.exchange.transactions[:])

for c in ['red', 'yellow', 'green', 'blue']:
    plt.plot(range(ohlc_train.shape[0]), ohlc_train[c], c=c)

# Evaluate

In [ ]:
def create_test_strategy(param_array):
    return create_strategy(data_manager_test, param_array)

In [ ]:
data_manager, exchange, strategy = create_test_strategy(best_params['x'])
strategy.trade()
strategy.stats('usdt').report()

In [ ]:
plot_transactions(ohlc_test, strategy.exchange.transactions[:])

for c in ['red', 'yellow', 'green', 'blue']:
    plt.plot(range(ohlc_test.shape[0]), ohlc_test[c], c=c)